We typically populate a data frame from some external source, the most common being a CSV file.  For this we use the `read_csv` function from the pandas library.  We pass it a path to the file we want to load.  On this machine, we are running Linux in the background, so we use Linux-style paths.  Your path would look different on a Windows machine.  Advanced: If you want to run the same code on all computers, [you need a library to build your paths for you](https://medium.com/@ageitgey/python-3-quick-tip-the-easy-way-to-deal-with-file-paths-on-windows-mac-and-linux-11a072b58d5f)

In [2]:
import pandas as pd

box_hill = pd.read_csv("data/rainfall/box_hill.csv")
box_hill

,Product code,Bureau of Meteorology station number,Year,Month,Day,Rainfall amount (millimetres),Period over which rainfall was measured (days),Quality
0,IDCJAC0009,67104,1990,1,1,NaN,NaN,NaN
1,IDCJAC0009,67104,1990,1,2,NaN,NaN,NaN
2,IDCJAC0009,67104,1990,1,3,NaN,NaN,NaN
3,IDCJAC0009,67104,1990,1,4,NaN,NaN,NaN
4,IDCJAC0009,67104,1990,1,5,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
11764,IDCJAC0009,67104,2022,3,18,0.0,1.0,N
11765,IDCJAC0009,67104,2022,3,19,17.0,1.0,N
11766,IDCJAC0009,67104,2022,3,20,1.0,1.0,N
11767,IDCJAC0009,67104,2022,3,21,0.0,1.0,N



A number of very interesting things happen:
  1) The dataframe has been given an index that starts at 0 and goes up one at at time.  That data was not in the original CSV file.
  2) Any empty cells were given the value `NaN` (which means "not a number")
  3) The first row is used to create column names (remember a column in a `Series`)
  4) When printing the frame, only the first 5 and last 5 rows are shown and the full imensions are shown at the bottom.

If we extract one of the series from this frame, it will use the generated indexes.

In [2]:
box_hill["Rainfall amount (millimetres)"]

0         NaN
1         NaN
2         NaN
3         NaN
4         NaN
         ... 
11764     0.0
11765    17.0
11766     1.0
11767     0.0
11768     0.0
Name: Rainfall amount (millimetres), Length: 11769, dtype: float64

# Setting the index columns

Typically, the data will already have a column that works as an index.  An index is any column that is unique for that row.  I.e. it has a different value on every row of the data.  For this reason we don't have a single column in our data that can do this, but there is a combination that works!  If we combine "Year", "Month", and "Day", the result is different for each row.  The advantage of doing this is that the index becomes a more natural way to look at the data.  If we do use multiple columns for the index, we get a "Multi-Index" which we will talke about soon.

In [3]:
box_hill_multi = pd.read_csv("data/rainfall/box_hill.csv", index_col=["Year", "Month", "Day"])
box_hill_multi

Product code  Bureau of Meteorology station number  \
Year Month Day                                                      
1990 1     1     IDCJAC0009                                 67104   
           2     IDCJAC0009                                 67104   
           3     IDCJAC0009                                 67104   
           4     IDCJAC0009                                 67104   
           5     IDCJAC0009                                 67104   
...                     ...                                   ...   
2022 3     18    IDCJAC0009                                 67104   
           19    IDCJAC0009                                 67104   
           20    IDCJAC0009                                 67104   
           21    IDCJAC0009                                 67104   
           22    IDCJAC0009                                 67104   

                Rainfall amount (millimetres)  \
Year Month Day                                  
1990 1     1                              NaN   
           2                              NaN   
           3                              NaN   
           4                              NaN   
           5                              NaN   
...                                       ...   
2022 3     18                             0.0   
           19                            17.0   
           20                             1.0   
           21                             0.0   
           22                             0.0   

                Period over which rainfall was measured (days) Quality  
Year Month Day                                                          
1990 1     1                                               NaN     NaN  
           2                                               NaN     NaN  
           3                                               NaN     NaN  
           4                                               NaN     NaN  
           5                                               NaN     NaN  
...                                                        ...     ...  
2022 3     18                                              1.0       N  
           19                                              1.0       N  
           20                                              1.0       N  
           21                                              1.0       N  
           22                                              1.0       N  

[11769 rows x 5 columns]

That is really all there is to loading csv files, we will now look at more data frame techniques.  In particular, we will look at the types of things you often want to do when loading up a CSV file.

# More DataFrame techniques.

## Extracting a single value

We can extract a single value by using the square bracket notation twice.  For example, I can get the 11,000th value from the rainfall amount column like this.

In [4]:
box_hill["Rainfall amount (millimetres)"][11000]

19.0

## Working with multi-indexes

What should I put if I want to look up an index in a table with a multi-index?  First, lets pull a series from the data frame.  You will notice that it also has a multi-index.

In [5]:
box_hill_multi["Rainfall amount (millimetres)"]

Year  Month  Day
1990  1      1       NaN
             2       NaN
             3       NaN
             4       NaN
             5       NaN
                    ... 
2022  3      18      0.0
             19     17.0
             20      1.0
             21      0.0
             22      0.0
Name: Rainfall amount (millimetres), Length: 11769, dtype: float64

In fact, I can ask for one, two, or three indexes, depending on what data I want. I can drop off the year and reduce the series to months and days (a multi-index sized 2) by giving a year.  Notice that the resulting series has a multi-index!  This can trick people into thinking they are looking at a data frame (since it looks like there is more than one column) but hte missing column name on the last column and the print-out giving a `dtype` at the bottom are the giveaways that this is a series with a complex index instead of a frame with mutliple columns.

In [6]:
box_hill_multi["Rainfall amount (millimetres)"][1993]

Month  Day
1      1       0.0
       2       0.0
       3       0.0
       4      14.0
       5       4.0
              ... 
12     27      0.0
       28      0.0
       29      0.0
       30      1.8
       31      0.0
Name: Rainfall amount (millimetres), Length: 365, dtype: float64

 Or, I can get just days if I give a year and a month (say January 2021).  Notice that I need to put the two indexes into parenthesis.  This is called a "tuple" and many times you need to give multiple values in one place, you bundle them in this way.

In [7]:
box_hill_multi["Rainfall amount (millimetres)"][(2021,1)]

Day
1      1.0
2      0.0
3      1.0
4      0.0
5     13.0
6      0.0
7      4.0
8      5.0
9      0.0
10     0.0
11     0.0
12     0.0
13     0.0
14     0.0
15     0.0
16     0.0
17     0.0
18     0.0
19     0.0
20     0.0
21     0.0
22     0.0
23     0.0
24     0.0
25     0.0
26     0.0
27     0.0
28     1.0
29     4.0
30     5.0
31     3.0
Name: Rainfall amount (millimetres), dtype: float64

Or I can get right down to a single value if I give values for all three indexes (say 3rd Feb 2022).

In [8]:
box_hill_multi["Rainfall amount (millimetres)"][(2022,2,3)]

2.0

# Exercises

## Load without multi-indexes
Load the daily rainfall data for Lithgow (you will find it in the same folder as Box Hill).  What was the first year for which data is recorded in that file?  What was the last?  You can assume the data is sorted in order from oldest to newest.

In [9]:
# load data into variable `lithgow`

print("print the rainfall from first year within the lithgow data")

print("print the rainfall from last year within in the lithgow data")

print the first year within the lithgow data
print the last year within in the lithgow data


## Load with multi-indexes
Can you do the same for Hornsby Pool but by using multi-indexes (or not using them, if you used them above)?

## Particular month
Can you display the rainfall data for the month of March 2005 in a single table using what we have learned so far?

# Conclusion

Multi-Indexes is _not_ something Excel does well.  The move to pandas has paid off already!

# Futher Work


Pandas is [capable of reading data from a multitude of formats](https://medium.com/analytics-vidhya/tips-tricks-to-import-data-into-pandas-from-various-data-sources-in-your-next-data-science-12cd384a31ae).  We will take a look here at:
  
## Excel

Beyond the additional formatting, the primary difference between a CSV file and an Excel file is that the Excel file might have mutliple "worksheets", each one of which represents a table that can be imported into a DataFrame.  We have provided an example file with some fun data in `data/fun.xlxs`, here is how to read it using [pandas built-in `read_excel`](https://pandas.pydata.org/docs/reference/api/pandas.read_excel.html) function.

Note: you need to have an additional library installed to make this work:
  * [`openpyxl`](https://pythonexamples.org/modulenotfounderror-no-module-named-openpyxl/)

You might also note that reading these excel files is _very slow_  because they are very large datasets.

If we provide no extra arguments to `read_excel` it will only read the first sheet.

In [11]:
pd.read_excel("data/fun.xlsx")

,ID,Name,Sex,Age,Height,Weight,Team,NOC,Games,Year,Season,City,Sport,Event,Medal
0,1724,Aristidis Akratopoulos,M,NaN,NaN,NaN,Greece,GRE,1896 Summer,1896,Summer,Athina,Tennis,Tennis Men's Singles,NaN
1,1724,Aristidis Akratopoulos,M,NaN,NaN,NaN,Greece-3,GRE,1896 Summer,1896,Summer,Athina,Tennis,Tennis Men's Doubles,NaN
2,1725,"Konstantinos ""Kostas"" Akratopoulos",M,NaN,NaN,NaN,Greece,GRE,1896 Summer,1896,Summer,Athina,Tennis,Tennis Men's Singles,NaN
3,1725,"Konstantinos ""Kostas"" Akratopoulos",M,NaN,NaN,NaN,Greece-3,GRE,1896 Summer,1896,Summer,Athina,Tennis,Tennis Men's Doubles,NaN
4,4113,Anastasios Andreou,M,NaN,NaN,NaN,Greece,GRE,1896 Summer,1896,Summer,Athina,Athletics,Athletics Men's 110 metres Hurdles,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
271111,135528,Marc Zwiebler,M,32.0,181.0,75.0,Germany,GER,2016 Summer,2016,Summer,Rio de Janeiro,Badminton,Badminton Men's Singles,NaN
271112,135547,Viktoriya Viktorovna Zyabkina,F,23.0,174.0,62.0,Kazakhstan,KAZ,2016 Summer,2016,Summer,Rio de Janeiro,Athletics,Athletics Women's 100 metres,NaN
271113,135547,Viktoriya Viktorovna Zyabkina,F,23.0,174.0,62.0,Kazakhstan,KAZ,2016 Summer,2016,Summer,Rio de Janeiro,Athletics,Athletics Women's 200 metres,NaN
271114,135547,Viktoriya Viktorovna Zyabkina,F,23.0,174.0,62.0,Kazakhstan,KAZ,2016 Summer,2016,Summer,Rio de Janeiro,Athletics,Athletics Women's 4 x 100 metres Relay,NaN


We can index other sheets with a number (starting at 0 for the left-most sheet) or by name

In [ ]:
pd.read_excel("data/fun.xlsx", "vino")

,Unnamed: 0,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum SelecciÃ³n Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen RodrÃ­guez
2,2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley
3,3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi
4,4,France,"This is the top wine from La BÃ©gude, named af...",La BrÃ»lade,95,66.0,Provence,Bandol,NaN,Provence red blend,Domaine de la BÃ©gude
...,...,...,...,...,...,...,...,...,...,...,...
150925,150925,Italy,Many people feel Fiano represents southern Ita...,NaN,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Feudi di San Gregorio
150926,150926,France,"Offers an intriguing nose with ginger, lime an...",CuvÃ©e Prestige,91,27.0,Champagne,Champagne,NaN,Champagne Blend,H.Germain
150927,150927,Italy,This classic example comes from a cru vineyard...,Terre di Dora,91,20.0,Southern Italy,Fiano di Avellino,NaN,White Blend,Terredora
150928,150928,France,"A perfect salmon shade, with scents of peaches...",Grand Brut RosÃ©,90,52.0,Champagne,Champagne,NaN,Champagne Blend,Gosset



# Exercise - whr

Read in all three sheets (they are called `2015`, `2016`, and `2017`) from the `data/whr.xlsx`.  We've given a sneak-peak of next worksheet's material to combine the three into one data frame.

In [6]:
whr_2015 = # fixme
whr_2016 = # fixme
whr_2017 = # fixme

#pd.concat([whr_2015, whr_2016, whr_2017])
pd.concat([whr_2015,whr_2016, whr_2017])

,Country,Region,Happiness Rank,Happiness Score,Standard Error,Economy (GDP per Capita),Family,Health (Life Expectancy),Freedom,Trust (Government Corruption),...,Lower Confidence Interval,Upper Confidence Interval,Happiness.Rank,Happiness.Score,Whisker.high,Whisker.low,Economy..GDP.per.Capita.,Health..Life.Expectancy.,Trust..Government.Corruption.,Dystopia.Residual
0,Switzerland,Western Europe,1.0,7.587,0.03411,1.39651,1.349510,0.94143,0.665570,0.41978,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Iceland,Western Europe,2.0,7.561,0.04884,1.30232,1.402230,0.94784,0.628770,0.14145,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Denmark,Western Europe,3.0,7.527,0.03328,1.32548,1.360580,0.87464,0.649380,0.48357,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Norway,Western Europe,4.0,7.522,0.03880,1.45900,1.330950,0.88521,0.669730,0.36503,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Canada,North America,5.0,7.427,0.03553,1.32629,1.322610,0.90563,0.632970,0.32957,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,Rwanda,NaN,NaN,NaN,NaN,NaN,0.945707,NaN,0.581844,NaN,...,NaN,NaN,151.0,3.471,3.543030,3.398970,0.368746,0.326425,0.455220,0.540061
151,Syria,NaN,NaN,NaN,NaN,NaN,0.396103,NaN,0.081539,NaN,...,NaN,NaN,152.0,3.462,3.663669,3.260331,0.777153,0.500533,0.151347,1.061574
152,Tanzania,NaN,NaN,NaN,NaN,NaN,1.041990,NaN,0.390018,NaN,...,NaN,NaN,153.0,3.349,3.461430,3.236570,0.511136,0.364509,0.066035,0.621130
153,Burundi,NaN,NaN,NaN,NaN,NaN,0.629794,NaN,0.059901,NaN,...,NaN,NaN,154.0,2.905,3.074690,2.735310,0.091623,0.151611,0.084148,1.683024


# Exercise - mtb

Lets now load data that is not already in a neat table format.  You won't need anything we haven't spoken about, but it is interesting to see how it works.

Load the data in `wsmtb/2018-XC-Club-Champs`

In [ ]:
"fix me"

'fix me'

# Concept Summary
  * No new libraries required in this topic
  * We can extract a series from a data frame with `[]`
  * We can control which columns get used as an index.
  * Index is a key concept for understanding DataFrames and Series
  * All indexes, including multi-indexes, transfer to the series.
  
# Python Concepts
  * Multi-Indexes are tuples
  * Tuples are created wtih `(`
  * The `read_csv` function comes from the `pandas` module, which we imported as `pd`, so we need that prefix to call it.
  * We have now seen lists (`[]`), dictionaries (`{}`) and tuples (`()`). Have a read through [this exploration of different data forms from Python](https://www.educative.io/edpresso/list-vs-tuple-vs-set-vs-dictionary-in-python) and attempt the following exercises.  Note that `Series` and `DataFrames` are like extensions of tuples/dictionaries/lists and we like to use them whenver possible, but sometimes (as you have seen) we need the simpler forms.

In [ ]:
# Create a tuple to store your name, age, and whether or not you like Pizza

a1 = "fixme"

# Create a list with the numbers 1 to 10 in it

a2 = "fixme"

# Create a dictionary that has three keys, "bim", "bam", "bat" associated with values 1, 2, and 3 respectively

a3 = "fixme"

print(a1)
print(a2)
print(a3)

fixme
fixme
fixme
